In [ ]:
# First two lines should be commented out in very first run, uncomment after
# %cd ..
# !rm -rf /content/RL-Chess-Transformers/
!git clone https://github.com/EmilGou/RL-Chess-Transformers.git
!rm -rf /content/RL-Chess-Transformers/GRPO/colab_notebooks

%cd /content/RL-Chess-Transformers/

/content
Cloning into 'RL-Chess-Transformers'...
remote: Enumerating objects: 157, done.
remote: Counting objects: 100% (157/157), done.
remote: Compressing objects: 100% (111/111), done.
remote: Total 157 (delta 96), reused 104 (delta 44), pack-reused 0 (from 0)
Receiving objects: 100% (157/157), 3.09 MiB | 27.54 MiB/s, done.
Resolving deltas: 100% (96/96), done.
/content/RL-Chess-Transformers


In [ ]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
!pip install python-chess cairosvg
import torch
from torch import nn
import torch.nn.functional as F
import chess

In [ ]:
# download raw moves file
!pip install -U -q gdown
FILE_ID="1BSBuF2dKOnVWuR5CNjp-o7QBYb-10JTO"
!gdown --id $FILE_ID -O moves

/usr/local/lib/python3.11/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=1BSBuF2dKOnVWuR5CNjp-o7QBYb-10JTO
From (redirected): https://drive.google.com/uc?id=1BSBuF2dKOnVWuR5CNjp-o7QBYb-10JTO&confirm=t&uuid=64ac678e-5e60-4bcf-ad5d-9e5d91bfec7d
To: /content/RL-Chess-Transformers/moves
100% 134M/134M [00:00<00:00, 205MB/s]


In [ ]:
# Pretrained Model
# Dataset
# Generation Function
# GRPO Objective Function
  # Reward Function
    # Stockfish
# Training Loop
# Logging

In [ ]:
moves = open("moves", "r").read()
moves = moves.split('\n\n')[:-1]
GAMES = [m.split('\n')[:-1] for m in moves]

In [ ]:
from GRPO.model import AutoregressiveTransformer, ChessConfig

model = AutoregressiveTransformer(ChessConfig())
model


AutoregressiveTransformer(
  (token_emb): Embedding(2008, 512, padding_idx=2006)
  (pos_emb): Embedding(512, 512)
  (transformer): TransformerEncoder(
    (layers): ModuleList(
      (0-5): 6 x TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=512, out_features=512, bias=True)
        )
        (linear1): Linear(in_features=512, out_features=2048, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
        (linear2): Linear(in_features=2048, out_features=512, bias=True)
        (norm1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.1, inplace=False)
        (dropout2): Dropout(p=0.1, inplace=False)
      )
    )
  )
  (lm_head): Linear(in_features=512, out_features=2008, bias=False)
)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

AutoregressiveTransformer(
  (token_emb): Embedding(2008, 512, padding_idx=2006)
  (pos_emb): Embedding(512, 512)
  (transformer): TransformerEncoder(
    (layers): ModuleList(
      (0-5): 6 x TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=512, out_features=512, bias=True)
        )
        (linear1): Linear(in_features=512, out_features=2048, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
        (linear2): Linear(in_features=2048, out_features=512, bias=True)
        (norm1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.1, inplace=False)
        (dropout2): Dropout(p=0.1, inplace=False)
      )
    )
  )
  (lm_head): Linear(in_features=512, out_features=2008, bias=False)
)

In [ ]:
from GRPO.data import RLThinkDataset

d = RLThinkDataset(GAMES)

In [ ]:
!   wget https://github.com/official-stockfish/Stockfish/releases/latest/download/stockfish-ubuntu-x86-64-sse41-popcnt.tar && \
    tar -xf stockfish-ubuntu-x86-64-sse41-popcnt.tar stockfish/stockfish-ubuntu-x86-64-sse41-popcnt


--2025-05-28 17:19:00--  https://github.com/official-stockfish/Stockfish/releases/latest/download/stockfish-ubuntu-x86-64-sse41-popcnt.tar
Resolving github.com (github.com)... 140.82.116.3
Connecting to github.com (github.com)|140.82.116.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/official-stockfish/Stockfish/releases/download/sf_17.1/stockfish-ubuntu-x86-64-sse41-popcnt.tar [following]
--2025-05-28 17:19:00--  https://github.com/official-stockfish/Stockfish/releases/download/sf_17.1/stockfish-ubuntu-x86-64-sse41-popcnt.tar
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://objects.githubusercontent.com/github-production-release-asset-2e65be/20976138/1f1e24b8-d3b0-49bf-92fd-ebd01964592a?X-Amz-Algorithm=AWS4-HMAC-SHA256&X-Amz-Credential=releaseassetproduction%2F20250528%2Fus-east-1%2Fs3%2Faws4_request&X-Amz-Date=20250528T171900Z&X-Amz-Expires=300&X-Amz-Signature=2aa773

In [ ]:
import chess, chess.engine, torch, numpy as np
from GRPO.utils import extract_fen_from_game
from GRPO.vocab import UCI_IDS, UCI_MOVES      # id → uci   and   uci → id

path = "stockfish/stockfish-ubuntu-x86-64-sse41-popcnt"

def _pad(seqs, pad_id):
    max_len = max(len(s) for s in seqs)
    return torch.tensor(
        [s + [pad_id] * (max_len - len(s)) for s in seqs],
        dtype=torch.long,
    )

def _generate_completions_and_score(
    model,
    prompt_ids,                  # (B, T)
    engine_path     = path,
    depth           = 12,        # for Stockfish analyse
    num_generations = 4,
    num_moves       = 10,
    limit = 5
):
    device, pad_id = prompt_ids.device, model.pad_id

    # 1) repeat prompts: (B, T) → (B·G, T)
    B, T       = prompt_ids.shape
    input_ids  = prompt_ids.repeat_interleave(num_generations, dim=0)
    batch_size = input_ids.size(0)                 # B·G
    seqs       = input_ids.tolist()

    # 2) boards
    boards = [chess.Board(extract_fen_from_game(s)) for s in seqs]

    # 3) Stockfish eval before roll‑outs
    engine = chess.engine.SimpleEngine.popen_uci(engine_path)
    with engine:
        base_eval = torch.tensor(
            [
                engine.analyse(b, chess.engine.Limit(depth=depth))["score"]
                      .pov(b.turn).score(mate_score=10000)
                for b in boards
            ],
            dtype=torch.float32, device=device
        )

        # 4) roll‑out:  num_moves × 2 half‑moves
        completions, completion_masks = [[] for _ in range(batch_size)], [[] for _ in range(batch_size)]

        for _ in range(num_moves):
            # 4‑a) *policy* move  (mask = 1)
            move_ids, input_ids = model.predict_move(
                seqs,
                _pad(seqs, pad_id).to(device),
                boards,
            )
            for i, tok in enumerate(move_ids):
                completions[i].append(tok)
                completion_masks[i].append(1 if tok != pad_id else 0)
                if tok != pad_id and not boards[i].is_game_over():
                    boards[i].push_uci(UCI_IDS[tok])
                seqs[i].append(tok)

            # 4‑b) *model* response move  (mask = 0)
            resp_ids, input_ids = model.predict_move(
                seqs,
                _pad(seqs, pad_id).to(device),
                boards,
            )
            for i, tok in enumerate(resp_ids):
                completions[i].append(tok)
                completion_masks[i].append(0)           # never optimised
                if tok != pad_id and not boards[i].is_game_over():
                    boards[i].push_uci(UCI_IDS[tok])
                seqs[i].append(tok)

        # 5) Stockfish eval after roll‑outs
        after_eval = torch.tensor(
            [
                engine.analyse(b, chess.engine.Limit(depth=depth))["score"]
                      .pov(b.turn).score(mate_score=10000)
                for b in boards
            ],
            dtype=torch.float32, device=device
        )

    # 6) centred rewards → advantages
    delta        = (after_eval - base_eval) / 100.0
    scaled_reward = torch.clamp(delta, -limit, limit)
    # ------------------------------------------------------------------
#  ⬇  NEW: per‑token advantages (B·G, L)  --------------------------
# ------------------------------------------------------------------
    rewards   = scaled_reward.view(B, num_generations)          # (B, G) as before
    centered  = rewards - rewards.mean(dim=1, keepdim=True)     # baseline

    # Expand to (B·G, L) by repeating each reward over its two half‑moves
    # (first is policy, second is response) …
    expanded = centered.repeat_interleave(num_moves * 2, dim=1)   # (B, G*L)
    expanded = expanded.view(-1, num_moves * 2)                    # (B·G, L)

    # Cumulative future return per token (process supervision)
    advantages = torch.flip(torch.cumsum(torch.flip(expanded, [1]), dim=1), [1])




    # 7) pad everything for the backward pass
    input_ids  = _pad(seqs, pad_id).to(device)                          # (B·G, T+L)
    max_L      = max(len(c) for c in completions)
    completion_ids  = torch.full((batch_size, max_L), pad_id,
                                 dtype=torch.long, device=device)
    completion_mask = torch.zeros_like(completion_ids, dtype=torch.float32)
        # Zero‑out tokens that are masked out (responses & padding)

    for i, (c, m) in enumerate(zip(completions, completion_masks)):
        L = len(c)
        completion_ids[i, :L]  = torch.tensor(c, dtype=torch.long, device=device)
        completion_mask[i, :L] = torch.tensor(m, dtype=torch.float32, device=device)

    advantages = advantages * completion_mask                     # (B·G, L)


    return {
        "input_ids"       : input_ids,        # (B·G, T+L)
        "completion_ids"  : completion_ids,   # (B·G, L)
        "completion_mask" : completion_mask,  # (B·G, L)
        "advantages"      : advantages,       # (B·G,)
    }


In [ ]:
test_loader = torch.utils.data.DataLoader(d, batch_size=8, shuffle=True)

test_batch = next(iter(test_loader))
test_batch.shape

torch.Size([8, 128])

In [ ]:
a = torch.randn((5,5))
b = torch.randn((5, 2))

torch.cat([a,b], dim = 1).shape

torch.Size([5, 7])

In [ ]:
from GRPO.grpo_trainer import GRPOArgs, GRPOTrainer
ref_model = AutoregressiveTransformer(ChessConfig()).to(device)
trainer = GRPOTrainer(model, ref_model, GRPOArgs())

In [ ]:
outputs = trainer._generate_completions_and_score(model, test_batch.to(device), path)
loss = trainer._compute_loss(model, outputs)

In [ ]:
outputs["advantages"]

tensor([[ 40.0000,   0.0000,  39.8892,   0.0000,  39.2876,   0.0000,  38.0905,
           0.0000,  36.1762,   0.0000,  33.4037,   0.0000,  29.6099,   0.0000,
          24.6066,   0.0000,  18.1766,   0.0000,  10.0701,   0.0000],
        [-40.0000,  -0.0000, -39.8892,  -0.0000, -39.2876,  -0.0000, -38.0905,
          -0.0000, -36.1762,  -0.0000, -33.4037,  -0.0000, -29.6099,  -0.0000,
         -24.6066,  -0.0000, -18.1766,  -0.0000, -10.0701,  -0.0000],
        [ 40.0000,   0.0000,  39.8892,   0.0000,  39.2876,   0.0000,  38.0905,
           0.0000,  36.1762,   0.0000,  33.4037,   0.0000,  29.6099,   0.0000,
          24.6066,   0.0000,  18.1766,   0.0000,  10.0701,   0.0000],
        [-40.0000,  -0.0000, -39.8892,  -0.0000, -39.2876,  -0.0000, -38.0905,
          -0.0000, -36.1762,  -0.0000, -33.4037,  -0.0000, -29.6099,  -0.0000,
         -24.6066,  -0.0000, -18.1766,  -0.0000, -10.0701,  -0.0000],
        [ 19.3000,   0.0000,  19.2465,   0.0000,  18.9563,   0.0000,  18.3787,
         

In [ ]:
# import torch, chess, random
# from pprint import pprint

# # ---------------------------------------------------------------------
# # 1.  A minimal stub policy
# # ---------------------------------------------------------------------
# PAD_ID = 0          # choose any value that *isn't* a legal‑move id

# try:
#     from GRPO.vocab import UCI_MOVES, UCI_IDS
# except ImportError:
#     # quick‑n‑dirty fall‑backs for testing outside your repo
#     UCI_MOVES, UCI_IDS = {}, {}

# def _register(move):
#     """Add move to the two lookup dicts if unseen; return its id."""
#     if move not in UCI_MOVES:
#         new_id = max(UCI_MOVES.values(), default=PAD_ID) + 1
#         UCI_MOVES[move] = new_id
#         UCI_IDS[new_id] = move
#     return UCI_MOVES[move]


# # ---------------------------------------------------------------------
# # 2.  Tiny prompts: one token per prompt is fine for a smoke test
# # ---------------------------------------------------------------------
# BATCH = test_batch.shape[0]
# PROMPT_LEN = 1
# prompt_ids = test_batch.to(device)

# # ---------------------------------------------------------------------
# # 3.  Run the rollout
# # ---------------------------------------------------------------------
# # IMPORTANT: make sure the function we’re testing is already imported.

# result = _generate_completions_and_score(
#     model            = model,
#     prompt_ids       = prompt_ids,
#     num_generations  = 2,      # keep tiny for speed
#     num_moves        = 3,      # six half‑moves per game
#     depth            = 4,      # shallow Stockfish eval → fast
# )

# # ---------------------------------------------------------------------
# # 4.  Pretty‑print the outputs
# # ---------------------------------------------------------------------
# print("\n=== Keys returned ===")
# print(list(result.keys()))

# for k, v in result.items():
#     print(f"\n--- {k} ---")
#     print(f"shape: {tuple(v.shape)}  dtype: {v.dtype}")
#     # show first sample only, otherwise it’s a wall of numbers
#     print("first row:", v[0].tolist() if v.ndim > 1 else v.tolist())

# # ---------------------------------------------------------------------
# # 5.  Sanity assertions
# # ---------------------------------------------------------------------
# input_ids       = result["input_ids"]
# completion_ids  = result["completion_ids"]
# completion_mask = result["completion_mask"]
# advantages      = result["advantages"]

# B, G = BATCH, 2
# assert input_ids.size(0) == completion_ids.size(0) == completion_mask.size(0) == advantages.size(0) == B * G, \
#        "batch dimension mismatch"

# for row_m, row_c in zip(completion_mask, result["completion_ids"]):
#     # figure out how many *real* half‑moves preceded the first pad
#     if 0 in row_c:
#         L = (row_c != pad_id).nonzero(as_tuple=False)[-1].item() + 1
#     else:
#         L = row_c.size(0)             # no pad at all

#     policy_bits   = row_m[:L:2]        # 0,2,4,...
#     response_bits = row_m[1:L:2]       # 1,3,5,...

#     assert policy_bits.bool().all(),      "even half‑moves must be 1"
#     if response_bits.numel():             # may be empty if L is odd
#         assert not response_bits.bool().any(), "odd half‑moves must be 0"


# print("\nAll tests passed ✅")


In [ ]:
from GRPO.grpo_trainer import GRPOArgs, GRPOTrainer
ref_model = AutoregressiveTransformer(ChessConfig()).to(device)
trainer = GRPOTrainer(model, ref_model, GRPOArgs())

In [ ]:

train_loader = torch.utils.data.DataLoader(d, batch_size=8, shuffle=True)
trainer.train(train_loader, path)

step      5 | loss   0.0091
step     10 | loss   0.0097
step     15 | loss   0.0081
step     20 | loss   0.2043
step     25 | loss   0.0085
step     30 | loss   0.0090
step     35 | loss   0.0095
step     40 | loss   0.0094
step     45 | loss   0.0114
step     50 | loss   0.2876
step     55 | loss   0.1387
step     60 | loss   0.0094
step     65 | loss   0.0107
step     70 | loss   0.2189
step     75 | loss   0.0098
step     80 | loss   0.1781
step     85 | loss   0.0080
step     90 | loss   0.0091
step     95 | loss   0.0100
step    100 | loss   0.0105
step    105 | loss   0.0090
step    110 | loss   0.0100
step    115 | loss   0.0087
step    120 | loss   0.0108
step    125 | loss   0.0085
step    130 | loss   0.5018
step    135 | loss   0.0093
step    140 | loss   0.0279
step    145 | loss   0.0112
step    150 | loss   0.0097
step    155 | loss   0.0101
step    160 | loss   0.0105
step    165 | loss   0.0097
step    170 | loss   0.0095
step    175 | loss   0.0098
step    180 | loss  

KeyboardInterrupt: 

In [ ]:
trainer._metrics["train"]["clip_ratio/high_mean"]

[0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0